In [5]:
# -------------------------------------
# Import necessary tools from Unified Planning
# -------------------------------------
from unified_planning.io.pddl_reader import PDDLReader
from unified_planning.shortcuts import OneshotPlanner
from unified_planning.engines import PlanGenerationResultStatus, OptimalityGuarantee
from unified_planning.shortcuts import get_environment

# Disable printing of planning engine credits
get_environment().credits_stream = None

# -------------------------------------
# 1) Define the PDDL domain as a string
# -------------------------------------
domain_pddl = '''(define (domain openrouter)
  (:requirements :typing :fluents :numeric-fluents)
  (:types
    llm provider account request capability
  )
  (:predicates
    (provided-by ?l - llm ?p - provider)
    (supports-capability ?l - llm ?c - capability)
    (requires-capability ?r - request ?c - capability)
    (request-owned-by ?r - request ?a - account)
    (routed ?r - request)
  )
  (:functions
    (remaining-budget ?a - account)
    (max-tokens ?l - llm)
    (tokens-required ?r - request)
    (cost-request ?l - llm ?r - request)
  )
  (:action route-request
    :parameters (?r - request ?l - llm ?p - provider ?a - account ?c - capability)
    :precondition (and
      (provided-by ?l ?p)
      (requires-capability ?r ?c)
      (supports-capability ?l ?c)
      (request-owned-by ?r ?a)
      (>= (remaining-budget ?a) (cost-request ?l ?r))
      (<= (tokens-required ?r) (max-tokens ?l))
    )
    :effect (and
      (routed ?r)
      (decrease (remaining-budget ?a) (cost-request ?l ?r))
    )
  )
)'''

# -------------------------------------
# 2) Define the PDDL problem as a string
# -------------------------------------
problem_pddl = '''(define (problem route-example)
  (:domain openrouter)
  (:objects
    openai-provider azure-provider - provider
    gpt4 llm-small               - llm
    acct-A                       - account
    code-cap multilingual-cap design-cap   - capability
    req1                         - request
  )
  (:init
    (provided-by gpt4 openai-provider)
    (provided-by llm-small azure-provider)

    (supports-capability gpt4 code-cap)
    (supports-capability gpt4 multilingual-cap)
    (supports-capability llm-small multilingual-cap)
    (supports-capability llm-small design-cap)

    (requires-capability req1 design-cap)

    (request-owned-by req1 acct-A)

    (= (remaining-budget acct-A)    1000)
    (= (max-tokens gpt4)            2000)
    (= (max-tokens llm-small)       1000)
    (= (tokens-required req1)       500)
    (= (cost-request gpt4 req1)     300)
    (= (cost-request llm-small req1) 200)
  )
  (:goal (and
    (routed req1)
  ))
)'''

# -------------------------------------
# 3) Write PDDL domain and problem to files
# -------------------------------------
with open('openrouter-domain.pddl', 'w') as f:
    f.write(domain_pddl)

with open('openrouter-problem.pddl', 'w') as f:
    f.write(problem_pddl)

# -------------------------------------
# 4) Load and solve the planning problem using TAMER
# -------------------------------------
reader = PDDLReader()
problem = reader.parse_problem('openrouter-domain.pddl', 'openrouter-problem.pddl')

with OneshotPlanner(
    name='tamer',
    problem_kind=problem.kind,
    optimality_guarantee=OptimalityGuarantee.SATISFICING
) as planner:
    try:
        result = planner.solve(problem)

        # -------------------------------------
        # 5) Output the plan
        # -------------------------------------
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print("✅ Plan found:")
            for action in result.plan.actions:
                print(f"  {action}")
        else:
            print(f"❌ No plan found. Reason: {result.status.name}")
            if result.log_messages:
                print("\nPlanner logs:")
                for msg in result.log_messages:
                    print(msg)

    except Exception as e:
        print(f"🚨 Planner error: {str(e)}")


✅ Plan found:
  route-request(req1, llm-small, azure-provider, acct-a, design-cap)
